## reuters 
The Reuters Corpus contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 topics, and grouped into two sets, called "training" and "test"; thus, the text with fileid 'test/14826' is a document drawn from the test set. This split is for training and testing algorithms that automatically detect the topic of a document, as we will see in chap-data-intensive.

In [1]:
from nltk.corpus import reuters
# len(reuters.fileids())
# reuters.fileids() #gets each file

In [2]:
type(reuters.words('test/14826'))

nltk.corpus.reader.util.StreamBackedCorpusView

#### calculating no. of grams

In [23]:
from collections import defaultdict
from nltk import ngrams
from funcy import flatten, isa

unigram=defaultdict(int)
bigram=defaultdict(int)
trigram=defaultdict(int)
fourgram=defaultdict(int)
fivegram=defaultdict(int)

def gram_calc(sentence,gram):
    if gram==1:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            unigram[gram]+=1
        return unigram
    elif gram==2:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            bigram[gram]+=1
        return bigram
    elif gram==3:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            trigram[gram]+=1
        return trigram
    elif gram==4:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            fourgram[gram]+=1
        return fourgram
    elif gram==5:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            fivegram[gram]+=1
        return fivegram

def start_stop_adder(news):
    sentence=[]
    temp=[]
    for word in news:
        # print(word)
        if word!=".":
            temp.append(word)
        elif word==" ":
            continue
        else:
            try:
                temp[0]="<s>"
                temp[len(temp)-1]="</s>"
            except IndexError:
                pass
            sentence.append([" ".join(temp)])
            temp=[]
    return sentence

for news in reuters.fileids():
    lowered_news = [each_string.lower() for each_string in reuters.words(news)]
    sentences=start_stop_adder(lowered_news)
    # print(sentences)
    para=[]
    for sentence in sentences:
        para.append(sentence[0].split(" "))
    # print("para",para)
    para = list(flatten(para))
    # print(para)
    gram_calc(para,1)
    gram_calc(para,2)
    gram_calc(para,3)
    gram_calc(para,4)
    gram_calc(para,5)

    

#### calculating probabilities

In [24]:
uni_count=0
for key,value in unigram.items():
    uni_count+=value
print(uni_count)

bi_count=0
for key,value in bigram.items():
    bi_count+=value
print(bi_count)

tri_count=0
for key,value in trigram.items():
    tri_count+=value
print(tri_count)

four_count=0
for key,value in fourgram.items():
    four_count+=value
print(four_count)

five_count=0
for key,value in fivegram.items():
    five_count+=value
print(five_count)

1569202
1559452
1549702
1539961
1530223


#### conditional probability : <br>
Bigram   : p(b|a)=p(a∩b)/p(a)<br>
trigram  : P(A| B,C) =P(A∩B∩C) /P(B∩C)<br>
fourgram : P(A| B,C,D) =P(A∩B∩C∩D) /P(B∩C∩D)<br>
fivegram : P(A| B,C,D,E) =P(A∩B∩C∩D∩E) /P(B∩C∩D∩E)<br>

In [25]:
uniprob=defaultdict(int)
for key, value in unigram.items():
    new_value = value/uni_count
    uniprob[key] = new_value

In [26]:
biprob=defaultdict(int)
for key, value in bigram.items():
    temp=[]
    temp.append(f'{key[1]}')
    temp=tuple(temp)
    prob_val=value/unigram[temp]
    biprob[key]=prob_val 
 

In [27]:
triprob=defaultdict(int)
for key, value in trigram.items():
    temp=key[1:3]
    prob_val=value/bigram[temp]
    triprob[key]=prob_val 

In [28]:
fourprob=defaultdict(int)
for key,value in fourgram.items():
    temp=key[1:4]
    prob_val=value/trigram[temp]
    fourprob[key]=prob_val
# four_prob

In [29]:
fiveprob=defaultdict(int)
for key,value in fivegram.items():
    temp=key[1:5]
    prob_val=value/fourgram[temp]
    fiveprob[key]=prob_val
# four_prob

In [30]:
biprob

defaultdict(int,
            {('<s>', 'exporters'): 0.07303370786516854,
             ('exporters', 'fear'): 0.0392156862745098,
             ('fear', 'damage'): 0.008264462809917356,
             ('damage', 'from'): 0.0009963880931622867,
             ('from', '</s>'): 0.016529012906360236,
             ('</s>', '<s>'): 0.8958338152629022,
             ('<s>', '.-'): 0.9090909090909091,
             ('.-', 'japan'): 0.014464168310322156,
             ('japan', 'rift'): 0.5,
             ('rift', 'mounting'): 0.045454545454545456,
             ('mounting', 'trade'): 0.0011152416356877324,
             ('trade', 'friction'): 0.5555555555555556,
             ('friction', 'between'): 0.0037914691943127963,
             ('between', 'the'): 0.0039314566295638904,
             ('the', '</s>'): 0.04859424177774022,
             ('</s>', '</s>'): 0.07898016518451237,
             ('<s>', 'japan'): 0.04339250493096647,
             ('japan', 'has'): 0.009009009009009009,
             ('has', 'r

#### using markov's assumption on chain rule of probability


#### prediction
n = how many words has to be predicted<br>
grams=how many grams does user wants us to predict the next word<br>

In [240]:
def predict(ip):
    ip=ip.split(" ")
    if len(ip)==2:
        for i in range(10):
            # print(ip)
            temp=[0]
            predictor=0.0
            for key,value in biprob.items():
                try:
                    if key[0]==ip[-1]:#checking first word is matching with the last word of our ip
                        if biprob[key]>predictor:
                            if key[-1]=="</s>":
                                break
                            else:
                                predictor=biprob[key]
                                temp[0]=(key,value)
                except TypeError:
                    pass
            try:
                ip.append(temp[0][0][-1])
            except TypeError: #if this occurs--> </s> is reached
                break #this will the first loop
                
        # print(ip)
        # print(temp)
        return " ".join(ip)
    if len(ip)==3:
        for i in range(12):
            # print(ip)
            temp=[0]
            predictor=0.0
            for key,value in triprob.items():
                try:
                    if key[0:2]==tuple(ip[-2:]):
                        # print("here",key[0:2],ip[-2:])
                        if triprob[key]>predictor:
                            if key[-1]=="</s>":
                                break
                            else:
                                predictor=triprob[key]
                                temp[0]=(key,value)
                except TypeError:
                    pass
            try:
                ip.append(temp[0][0][-1])
            except TypeError: #if this occurs--> </s> is reached
                break #this will the first loop
        return " ".join(ip)
    if len(ip)==4:
        for i in range(16): #random threshold values
            # print(ip)
            temp=[0]
            predictor=0.0
            for key,value in fourprob.items():
                try:
                    # print("in",key[0:3],tuple(ip[-3:]))
                    if key[0:3]==tuple(ip[-3:]):
                        print("in",key[0:3],tuple(ip[-3:]))
                        if fourprob[key]>predictor:
                            if key[-1]=="</s>":
                                break
                            else:
                                predictor=fourprob[key]
                                temp[0]=(key,value)
                except TypeError:
                    print("type error")
                    pass
            try:
                ip.append(temp[0][0][-1])
            except TypeError: #if this occurs--> </s> is reached
                break #this will the first loop
        return " ".join(ip)
    if len(ip)>=5:
        print("in")
        for i in range(3):
            # print(ip)
            temp=[0]
            predictor=0.0
            for key,value in fiveprob.items():
                try:
                    print(tuple(ip[-4:]))
                    if key[0:4]==tuple(ip[-4:]):
                        print(key,value)
                        if fiveprob[key]>predictor:

                            if key[-1]=="</s>":
                                print(key,value)
                                break
                            else:
                                predictor=fiveprob[key]
                                temp[0]=(key,value)
                except TypeError:
                    pass
            try:
                ip.append(temp[0][0][-1])
            except TypeError: #if this occurs--> </s> is reached
                break #this will the first loop
        return " ".join(ip)



In [241]:
print(predict("exporters fear tighter")) #predicting the next two words using bigram
print(predict("exporters fear")) #predicting the next two words using bigram

exporters fear tighter policy to deprive iranian rulers of oil equivalent at a proper level
exporters fear tighter monetary


In [239]:
print(predict("exporters fear damage from"))

in ('fear', 'damage', 'from') ('fear', 'damage', 'from')
exporters fear damage from


In [237]:
a=('fear', 'damage', 'from') 
b=('fears', 'damage', 'from')
if a==b:
    print("hey")

In [186]:
reuters.words('test/14828')

['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...]

In [ ]:
def predict(ip):
    

In [179]:
reuters.words("test/1000")

OSError: Zipfile 'C:\\Users\\rbw19\\AppData\\Roaming\\nltk_data\\corpora\\reuters.zip' does not contain 'reuters/test/1000'

In [108]:
ip=['exporters', 'fear']
temp=('fear', 'tighter')
new_temp=[]
for 


None


TypeError: 'NoneType' object is not iterable